In [1]:
import numpy as np
import sys
import os
import vector
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd


sys.path.append('../../utils/')
from DD_data_extractor_git import generate_random_data



In [2]:
class FakeParticleDataset(Dataset):


    def __len__(self):
        return 10000
    
    def __getitem__(self, idx):
        phi, eta, mass, pt = self.generate_input_data()
        px, py, pz, energy = self.generate_output_data(phi, eta, mass, pt)
        
        input_tensor = torch.tensor([phi, eta, mass, pt], dtype=torch.float32)
        output_tensor = torch.tensor([px, py, pz, energy], dtype=torch.float32)
        
        return input_tensor, output_tensor

    
    def generate_input_data(self):
        eta_low, eta_high = -2.5, 2.5
        mass_low, mass_high = 0, 11
        phi_low, phi_high = -np.pi, np.pi
        pt_low, pt_high = 0, 1000

        eta= np.random.uniform(eta_low, eta_high)
        mass = np.random.uniform(mass_low, mass_high)
        phi = np.random.uniform(phi_low, phi_high)
        pt = np.random.uniform(pt_low, pt_high)

        return phi, eta, mass, pt
    
    def generate_output_data(self, phi, eta, mass, pt):
        particle=vector.obj(pt=pt, phi=phi, eta=eta, mass=mass)

        px = particle.px
        py = particle.py
        pz = particle.pz
        energy = particle.e

        return px, py, pz, energy

def worker_init_fn(worker_id):
    numpy_seed = int(torch.initial_seed()) % (2**32 - 1)
    np.random.seed(numpy_seed + worker_id)

In [3]:
dataset= FakeParticleDataset()
train_loader = DataLoader(dataset, batch_size=320, shuffle=True, num_workers=4, worker_init_fn=worker_init_fn)
input_dim,output_dim=4,4

In [4]:
class CustomKinematicNet(nn.Module):
    def __init__(self, input_size, hidden_layers, lenoutput, activation_fn=F.relu):
        
        super(CustomKinematicNet, self).__init__()
        
        # Create the list of layers
        layers = [nn.Linear(input_size, hidden_layers[0])]
        for i in range(len(hidden_layers) - 1):
            layers.append(nn.Linear(hidden_layers[i], hidden_layers[i + 1]))
        layers.append(nn.Linear(hidden_layers[-1], lenoutput))
        
        self.layers = nn.ModuleList(layers)
        self.activation_fn = activation_fn
        
    def forward(self, x):
        for layer in self.layers[:-1]:
            x = self.activation_fn(layer(x))
        return self.layers[-1](x)

def custom_loss(y_pred, y_true):
    # print("y_pred:", y_pred.shape)
    se_loss = (y_pred - y_true) ** 2
    MSE_loss = torch.zeros_like(se_loss)  # Initialize with zeros
    
    num_features = int(output_dim)
    loss_list = []

    for i in range(num_features):
    
        RMSE = ((y_pred[:, i] - y_true[:, i]) / y_true[:, i]) ** 2
        mask = (y_true[:, i] > 1)
        
        RMSE_meanloss = torch.mean(RMSE[mask])
        MSE_meanloss = torch.mean(se_loss[:, i][~mask])

        # Weighted contribution of the RMSE for the masked values to the final loss tensor
        MSE_loss[:, i] = RMSE * mask.float()

        loss_list.append(MSE_meanloss.item())
        loss_list.append(RMSE_meanloss.item())
        
    full_loss = torch.mean(MSE_loss)  # Calculate the final average loss
    return loss_list, full_loss


hidden_layers = [16 for i in range(10)]
model = CustomKinematicNet(input_dim, hidden_layers, output_dim, activation_fn=F.tanh)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

CustomKinematicNet(
  (layers): ModuleList(
    (0): Linear(in_features=4, out_features=16, bias=True)
    (1-9): 9 x Linear(in_features=16, out_features=16, bias=True)
    (10): Linear(in_features=16, out_features=4, bias=True)
  )
)

In [11]:
epochs=1000

dfpath='/home/ddemler/HNLclassifier/saved_files/fnn_featregr/single_particle/losses1.csv'
dfcols=["train_loss","px_mse", "px_rmse", "py_mse", "py_rmse", "pz_mse", "pz_rmse", "energy_mse", "energy_rmse"]
losses_df=pd.DataFrame(columns=dfcols)

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)

        optimizer.zero_grad()
        outputs = model(data)

        loss_list, loss = custom_loss(outputs, target)
        loss.backward()
        
        optimizer.step()
        total_loss += loss.item()

    total_loss /= len(train_loader.dataset)
    loss_feats=np.array(loss_list)

    losses_full=np.array([total_loss]+list(loss_feats))

    losses_df.loc[epoch]=losses_full
    # print("loss feats before mean:", loss_feats.shape)
    # loss_feats=np.mean(loss_feats, axis=0)
    # print("loss feats after mean:", loss_feats.shape)
    pd.DataFrame(losses_df).to_csv(dfpath, index=False)
    print("Epoch: {}, Loss: {:.4e}, px_MSE: {:.3f}, px_RMSE: {:.3f}, py_MSE: {:.3f}, py_RMSE: {:.3f}, pz_MSE: {:.3f}, pz_RMSE: {:.3f}, energy_MSE: {:.3f}, energy_RMSE: {:.3f}".format(epoch, total_loss*10000, loss_feats[0], loss_feats[1], loss_feats[2], loss_feats[3], loss_feats[4], loss_feats[5], loss_feats[6], loss_feats[7]))

Epoch: 0, Loss: 1.6880e+01, px_MSE: 102224.562, px_RMSE: 0.898, py_MSE: 175362.344, py_RMSE: 0.871, pz_MSE: 1334764.125, pz_RMSE: 0.972, energy_MSE: nan, energy_RMSE: 0.763
Epoch: 1, Loss: 1.6721e+01, px_MSE: 196479.047, px_RMSE: 0.893, py_MSE: 155683.000, py_RMSE: 0.880, pz_MSE: 2154951.500, pz_RMSE: 0.928, energy_MSE: nan, energy_RMSE: 0.731
Epoch: 2, Loss: 1.6922e+01, px_MSE: 162337.453, px_RMSE: 0.872, py_MSE: 162767.984, py_RMSE: 0.856, pz_MSE: 2500755.000, pz_RMSE: 0.941, energy_MSE: nan, energy_RMSE: 0.734
Epoch: 3, Loss: 1.6937e+01, px_MSE: 173089.250, px_RMSE: 0.899, py_MSE: 146262.297, py_RMSE: 0.898, pz_MSE: 1391453.625, pz_RMSE: 0.940, energy_MSE: nan, energy_RMSE: 0.715
Epoch: 4, Loss: 1.6899e+01, px_MSE: 159262.453, px_RMSE: 0.888, py_MSE: 181254.453, py_RMSE: 0.895, pz_MSE: 2328493.500, pz_RMSE: 0.926, energy_MSE: nan, energy_RMSE: 0.753
Epoch: 5, Loss: 1.6813e+01, px_MSE: 220128.406, px_RMSE: 0.914, py_MSE: 182046.891, py_RMSE: 1.263, pz_MSE: 2600040.500, pz_RMSE: 0.922

Exception ignored in: <function _releaseLock at 0x7f06c4659820>
Traceback (most recent call last):
  File "/home/ddemler/.conda/envs/Dmitri-conda/lib/python3.8/logging/__init__.py", line 227, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 
IOStream.flush timed out
Exception ignored in: <function Socket.__del__ at 0x7f06c46a7dc0>
Traceback (most recent call last):
  File "/home/ddemler/.conda/envs/Dmitri-conda/lib/python3.8/site-packages/zmq/sugar/socket.py", line 178, in __del__
    def __del__(self):
KeyboardInterrupt: 


Epoch: 28, Loss: 1.6684e+01, px_MSE: 157707.297, px_RMSE: 0.864, py_MSE: 181090.062, py_RMSE: 0.841, pz_MSE: 2674998.500, pz_RMSE: 0.873, energy_MSE: nan, energy_RMSE: 0.750
Epoch: 29, Loss: 1.6479e+01, px_MSE: 199818.891, px_RMSE: 0.844, py_MSE: 148054.062, py_RMSE: 0.885, pz_MSE: 3214339.500, pz_RMSE: 0.898, energy_MSE: nan, energy_RMSE: 0.694


Epoch: 30, Loss: 1.6842e+01, px_MSE: 164177.578, px_RMSE: 0.910, py_MSE: 180510.031, py_RMSE: 1.149, pz_MSE: 2085591.875, pz_RMSE: 0.936, energy_MSE: nan, energy_RMSE: 0.702
